# <font color='green'><b> **Background subtracrtion** </b></font>


In this notebook, we present the background subtraction obtained using the mixture of Gaussian (MoG)

TUTORIAL: 
https://docs.opencv.org/3.4/d1/dc5/tutorial_background_subtraction.html

For testing, you can refer to the public dataset: http://scenebackgroundmodeling.net/

In [1]:
#@title ▶️ Base dir setup
import os, sys

# check if hosted (Google VM) or running on local server
if 'google.colab' in sys.modules:
  #@markdown Google Drive root folder - hosted by Google VM (adapt to your local paths)
  from google.colab import drive
  drive.mount('/content/drive', force_remount=False)
  base_dir = 'infoMM/' #@param {type: "string"}
  base_dir  = os.path.join('/content/drive/MyDrive/', base_dir)
  #!pip install pillow  --upgrade
  img_dir = 'data/img/'
  vid_dir = 'data/video/'
  out_dir = 'output/'
  
  # move to base_dir 
  os.chdir(base_dir)
else:
  #@markdown Path to local folder on PC (adapt to your local paths)
  img_dir = '../IMMAGINI/IMMAGINI_IN/'
  vid_dir = 'VIDEO_IN/'
  out_dir = 'VIDEO_OUT/'

In [2]:
import cv2
import numpy as np 
import matplotlib.pyplot as plt
import os 
import time
from ipywidgets import interact
import ipywidgets as widgets

In [3]:
#We load a sequence of images
capture = cv2.VideoCapture(vid_dir + 'IPPR/in000%3d.jpg')
nFrames = capture.get(cv2.CAP_PROP_FRAME_COUNT)
print("Loaded "+ str(nFrames)+ " images")

Loaded 299.0 images


In [4]:
#visualization of image sequence
def showVideo(I):
  
  n = len(I)

  def view_image(idx):
    plt.imshow(I[idx-1], interpolation='nearest', cmap='gray')
  
  interact(view_image, idx=widgets.IntSlider(min=1, max=n, step=1, value=1))

## <font color='green'><b> **- Background via  Mean Filter** </b></font>

This algorithm compute the  background as the mean of the last $N$ frames

In [5]:
def denoise(frame):
    frame1 = cv2.medianBlur(frame,5)
    frame2 = cv2.GaussianBlur(frame1,(5,5),0)
    
    return frame2

In [6]:
def computeMeanBackground(frames):
    
    # Convert the list of frames to a NumPy array
	frames_array = np.array(frames)

	# Compute the mean frame along the frames (axis=0)
	backGroundModel = np.mean(frames_array, axis=0)
 
	return backGroundModel

In [7]:
capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
start_time = time.time()

#INITIALIZATION of the parameter and background
I = [] #list to save the Foreground result, frame by frame for interactive visualization
frames = [] #list of the last N frames to be used for the background construction
N = 15 #temporal buffer length

binThresh = 15 #threshold to detect foreground 
count = 0

# Initialize the buffer prende i 15 frame precedenti
while count < N:
    ret, frame = capture.read()
    count += 1
    frame = denoise(frame)
    frames.append(frame)
    
# Initialize the BG model
backGroundModel = computeMeanBackground(frames)
 
# LOOP OVER all the frames, updating the model and working out the foreground
ret, frame = capture.read()

while ret:
     
  frame = denoise(frame)
  frames.pop(0)
  frames.append(frame)
  
  backGroundModel = computeMeanBackground(frames)
  foreGround = cv2.absdiff(backGroundModel.astype(np.uint8), frame)
  _, fgMask = cv2.threshold(foreGround, binThresh, 255, cv2.THRESH_BINARY)
  fgMaskBin = fgMask.max(2)
  I.append(fgMaskBin)
  ret, frame = capture.read() 
  
 
end_time = time.time() 
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

showVideo(I)  

Execution time: 5.661295175552368 seconds


interactive(children=(IntSlider(value=1, description='idx', max=284, min=1), Output()), _dom_classes=('widget-…

## <font color='green'><b> **- Exercise 1: Temporal *Median* Filter** </b></font>

Exploiting the above code of the mean filter to implement  the function `def computeMedianBackground(frames):` to get the Temporal *Median* Filter.


Then, adjust the main function to enable the invocation of either computeMeanBackground or computeMedianBackground based on a specified flag

In [8]:
def computeMedianBackground(frames): 
     # Convert the list of frames to a NumPy array
	frames_array = np.array(frames)

	# Compute the mean frame along the frames (axis=0)
	backGroundModel = np.median(frames_array, axis=0)
 
	return backGroundModel

In [9]:

Mean = 0 #flag: 0: Tempporal Median Filter; 1: Temporal Mean Filter

capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
start_time = time.time()

#INITIALIZATION of the parameter and background
I = [] #list to save the Foreground result, frame by frame for interactive visualization
frames = [] #list of the last N frames to be used for the background construction
N = 15 #temporal buffer length

binThresh = 15 #threshold to detect foreground 
count = 0

# Initialize the buffer prende i 15 frame precedenti
while count < N:
    ret, frame = capture.read()
    count += 1
    frame = denoise(frame)
    frames.append(frame)
    
# Initialize the BG model
if Mean == 0:
    backGroundModel = computeMedianBackground(frames)
else:
    backGroundModel = computeMeanBackground(frames)
# LOOP OVER all the frames, updating the model and working out the foreground
ret, frame = capture.read()

while ret:
     
  frame = denoise(frame)
  frames.pop(0)
  frames.append(frame)
  
  backGroundModel = computeMeanBackground(frames)
  foreGround = cv2.absdiff(backGroundModel.astype(np.uint8), frame)
  _, fgMask = cv2.threshold(foreGround, binThresh, 255, cv2.THRESH_BINARY)
  fgMaskBin = fgMask.max(2)
  I.append(fgMaskBin)
  ret, frame = capture.read() 
  
 
end_time = time.time() 
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

showVideo(I)  

Execution time: 5.919524669647217 seconds


interactive(children=(IntSlider(value=1, description='idx', max=284, min=1), Output()), _dom_classes=('widget-…

## <font color='green'><b> **- Background via online Mean Filter** </b></font>

This algorithm updates the background with learning rate $\alpha$.
 
It's a sort of mean filter since the oldest frames are progressively forgotten

In [10]:
def updateBackground(backGroundModel, alpha, frame):
		# apply the background averaging formula:
		# NEW_BACKGROUND = CURRENT_FRAME * ALPHA + OLD_BACKGROUND * (1 - APLHA)
		newbackGroundModel =  frame * alpha +  backGroundModel * (1 - alpha)

		# after the previous operation, the dtype of
		# self.backGroundModel will be changed to a float type
		# therefore we do not pass it to cv2.absdiff directly,
		# instead we acquire a copy of it in the uint8 dtype
		# and pass that to absdiff.

		return newbackGroundModel

In [12]:
#INITIALIZATION of the parameter and background

capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
nFrames = capture.get(cv2.CAP_PROP_FRAME_COUNT);
#print ("N frames to process: "+ str(nFrames))
start_time = time.time() 
#list to save the OF result, frame by frame for interactive visualization
I = []; 
alpha = 0.01 #learning rate
binThresh = 15 #threshold to detect foreground 
ret, frame = capture.read()
print(ret)
backGroundModel = denoise(frame);

# LOOP OVER all the frames, updating the model and working out the foreground
#for i in range(np.uint8(nFrames)-2):
while ret:
  frame = denoise(frame);
  backGroundModel = updateBackground(backGroundModel,alpha,frame);
  foreGround = cv2.absdiff(backGroundModel.astype(np.uint8), frame);
  _, fgMask = cv2.threshold(foreGround, binThresh, 255, cv2.THRESH_BINARY);
  fgMaskBin = fgMask.max(2)
  I.append(fgMaskBin);  
  ret, frame = capture.read();
 
end_time = time.time() 
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds") 

showVideo(I) 

True
Execution time: 8.724582433700562 seconds


interactive(children=(IntSlider(value=1, description='idx', max=299, min=1), Output()), _dom_classes=('widget-…

<font color='green'><b>QUESTION:</b></font> why in `fgMaskBin = fgMask.max(2)` we set 2? Read critically the code!

## <font color='green'><b> **- Exercise 2** </b></font>

Exploiting the above code of the online mean filter, implement  the function to get the *Running Gaussian Average*

In [13]:
#TODO

def updateBGgauss(mu, sigma2, alpha, frame):
  # apply the background averaging formula:
   #guardo le slide per le formule
  mu = frame * alpha + (1 - alpha) * mu

  sigma2 = alpha * (frame - mu)**2 + (1 - alpha) * alpha**2
   
  return (mu, sigma2)

In [14]:
# [GIVEN]
#INITIALIZATION of the parameter and background
capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
ret = True;
nFrames = capture.get(cv2.CAP_PROP_FRAME_COUNT);
print ("N frames to process: "+ str(nFrames))

#list to save the OF result, frame by frame for interactive visualization
I = []; 
alpha = 0.01; #learning rate
k = 3; #moltiple of sigma to determine the threshold

ret, frame = capture.read()
mu = denoise(frame);
sigma2 = np.zeros_like(frame) + 15; #at the beginning I set all the threshold to 15


N frames to process: 299.0


In [15]:
mu.shape

(218, 320, 3)

In [16]:
#TO DO

# LOOP OVER all the frames, updating the model and working out the foreground 
while ret:
  frame = denoise(frame);
  mu2, sigma3 = updateBGgauss(mu,sigma2, alpha, frame);
  foreGround = cv2.absdiff(mu2.astype(np.uint8), frame);
  _, fgMask = cv2.threshold(foreGround, binThresh, 255, cv2.THRESH_BINARY);
  fgMaskBin = fgMask.max(2)
  I.append(fgMaskBin);  
  ret, frame = capture.read();
 
end_time = time.time() 
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds") 

showVideo(I) 

Execution time: 182.82925057411194 seconds


interactive(children=(IntSlider(value=1, description='idx', max=299, min=1), Output()), _dom_classes=('widget-…

## <font color='green'><b> **- Exercise 3** </b></font>

Further improve the  *Running Gaussian Average* implementing the *selective background update* (Koller et al.)

$\mu_t = M \mu_{t-1} + (1-M)[\alpha I_t + (1-\alpha) \mu_{t-1}]$

$\sigma_{t}^2 = M \sigma_{t-1}^2 +  (1-M)[\alpha (I_t - \mu_t)^2 (1 - \alpha)\sigma_{t-1}^2]$

$M=0$ if $I_t$ is background 
$M=1$ if $I_t$ is foreground

## <font color='green'><b> **- Background subtraction using Mixture of Gaussians** </b></font>

**1. CREATE an object for generating the foreground mask based on the  Mixture of Gaussian Method**

`retval	=	cv.createBackgroundSubtractorMOG2(	[, history[, varThreshold[, detectShadows]]]	)`

**Parameters:**

- *history* : Length of the history.
- *varThreshold* : Threshold on the squared Mahalanobis distance between the pixel and the model to decide whether a pixel is well described by the background model. This parameter does not affect the background update. Highering this value increase the probability for a pixel to be included in the background
- *detectShadows* : If true, the algorithm will detect shadows and mark them. It decreases the speed a bit, so if you do not need this feature, set the parameter to false.


In [17]:
backSub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=10, detectShadows= False)
retval	=	cv2.BackgroundSubtractorMOG2.getNMixtures(backSub)
print("Default number of Gaussian in MoG: "+ str(retval))

#We can modify the Gaussian number
cv2.BackgroundSubtractorMOG2.setNMixtures(backSub, 3)	
retval	=	cv2.BackgroundSubtractorMOG2.getNMixtures(backSub)
print("Actual number of Gaussian in MoG: "+ str(retval))


Default number of Gaussian in MoG: 5
Actual number of Gaussian in MoG: 3


**2. At each frame the background is updated and the pixel classification done, using the function apply:**

`fgMask = backSub.apply(frame, learningRate)`

**Parameters**
- *frame*: input image
- *fgMask*: mask of the foreground
- *learningRate* (opt)=  The value between 0 and 1 that indicates how fast the background model is learnt. Negative parameter value makes the algorithm to use some automatically chosen learning rate. 0 means that the background model is not updated at all, 1 means that the background model is completely reinitialized from the last frame.

In [18]:
#list to save the OF result, frame by frame for interactive visualization
I = []; 

# Position the reader at the beginning of the video
capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
ret = True;

while ret:
    ret, frame = capture.read()
        
    #Every frame is used both for calculating the foreground mask and for updating the background. 
    #If you want to change the learning rate used for updating the background model, 
    #it is possible to set a specific learning rate by passing a parameter to the apply method.
    fgMask = backSub.apply(frame, 0.5)
    
    I.append(fgMask)
    #cv.imshow('Frame', frame)
    #cv.imshow('FG Mask', fgMask)
showVideo(I)      

interactive(children=(IntSlider(value=1, description='idx', max=300, min=1), Output()), _dom_classes=('widget-…

## <font color='green'><b> **- Exercise 4** </b></font>

Modify the parameters leariningRate (in "apply") and varThreshold to get better *results*